In [8]:
import pandas as pd
import defillama_utils as dfl
import requests as r
import plotly.express as px

chain = 'Optimism'
tokens = ['USDC','USDT','DAI','SUSD','LUSD','FRAX']
tokens_str = ', '.join(tokens)
print(tokens_str)

USDC, USDT, DAI, SUSD, LUSD, FRAX


In [9]:
# protos = 'https://api.llama.fi/protocols'
# s = r.Session()
# #get all protocols
# resp = pd.DataFrame( s.get(protos).json() )[['category','slug','chainTvls']]
# display(resp)
# # extract the chain names
# # resp['chainTvls'] = resp['chainTvls'].apply(lambda x: list(x.keys()) )
# # # set a true/false if the array contains the chain we want
# # resp['contains_chain'] = resp['chainTvls'].apply(lambda x: chain_name in x)
# # # filter where we have a match
# # proto_list = resp[['slug']][resp['contains_chain'] == True]
# # # clean up
# # proto_list = proto_list.reset_index(drop=True)
# # # boom


In [10]:
slugs = dfl.get_protocols_by_chain(chain, exclude_cex = True, exclude_chain = True)
display(slugs)
pds_raw = dfl.get_range(slugs, chain)


,slug
0,curve
1,uniswap-v3
2,multichain
3,aave-v3
4,arrakis-finance
...,...
88,optinyan
89,pegasus-finance
90,juggler-red
91,bitant


In [11]:
pds = pds_raw.copy()
pds = pds[pds['token'].isin(tokens)]
pds['total_token_tvl'] = pds.groupby(['protocol', 'chain','date'])['usd_value'].\
                                transform('sum').astype(int)

pds['day_rank'] = pds.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
pds['usd_value_millions'] = (pds['usd_value'] / 1e6)
display(pds)

,date,token,token_value,usd_value,total_app_tvl,protocol,chain,category,total_token_tvl,day_rank,usd_value_millions
1,2022-07-17 00:00:00,FRAX,5.727694e+02,5.730323e+02,1.108283e+07,curve,Optimism,Dexes,11029294,160,5.730323e-04
2,2022-07-17 00:00:00,SUSD,2.415611e+06,2.434926e+06,1.108283e+07,curve,Optimism,Dexes,11029294,160,2.434926e+00
3,2022-07-17 00:00:00,LUSD,4.000000e-05,4.000000e-05,1.108283e+07,curve,Optimism,Dexes,11029294,160,4.000000e-11
4,2022-07-17 00:00:00,USDT,4.191069e+06,4.191538e+06,1.108283e+07,curve,Optimism,Dexes,11029294,160,4.191538e+00
5,2022-07-17 00:00:00,USDC,2.258077e+06,2.258332e+06,1.108283e+07,curve,Optimism,Dexes,11029294,160,2.258332e+00
...,...,...,...,...,...,...,...,...,...,...,...
814,2022-12-22 00:00:00,DAI,6.731930e+00,6.745400e+00,4.082945e+03,symphony-finance,Optimism,Dexes,3119,2,6.745400e-06
818,2022-12-22 15:59:47,SUSD,2.622760e+02,2.622760e+02,4.056701e+03,symphony-finance,Optimism,Dexes,3116,1,2.622760e-04
819,2022-12-22 15:59:47,USDT,1.528769e+03,1.528769e+03,4.056701e+03,symphony-finance,Optimism,Dexes,3116,1,1.528769e-03
821,2022-12-22 15:59:47,USDC,1.317280e+03,1.318597e+03,4.056701e+03,symphony-finance,Optimism,Dexes,3116,1,1.318597e-03


In [17]:
pds_latest = pds[pds['day_rank'] == 1]
#exclude if all usdt
pds_latest = pds_latest[~ ( (pds_latest['token'] == 'USDT') & (pds_latest['usd_value'] == pds_latest['total_app_tvl']) )]

pds_latest = pds_latest.sort_values(by = 'total_token_tvl', ascending=False)
display(pds_latest)

,date,token,token_value,usd_value,total_app_tvl,protocol,chain,category,total_token_tvl,day_rank,usd_value_millions


In [13]:
stable_sum = pds_latest[['protocol','usd_value','usd_value_millions']].groupby(['protocol']).sum()
stable_sum = stable_sum.reset_index()
display(stable_sum)

,protocol,usd_value,usd_value_millions
0,2pi-network,7.650480e+05,0.765048
1,aave-v3,6.858992e+07,68.589920
2,acryptos,4.651724e+05,0.465172
3,aelin,2.141183e+05,0.214118
4,alchemix,1.500948e+05,0.150095
...,...,...,...
72,woofi-earn,5.910168e+05,0.591017
73,xtoken,2.885590e+05,0.288559
74,xy-finance,1.510465e+05,0.151046
75,yearn-finance,1.271910e+05,0.127191


In [14]:
# stable_sum['usd_value_millions'] = stable_sum['usd_value_millions'].map("${:,.2f}M".format)
chart_pds_latest = pds_latest[pds_latest['total_token_tvl']>1e6]
fig = px.bar(chart_pds_latest, x="protocol", y="usd_value", color="token", title="Protocol TVL by Stablecoin - > $1M in Selected Tokens\
        <br><sup>Note: if the bar is 100% USDT, that means DefiLlama doesn't have splits by token</sup>")
fig.update_layout(barmode='stack', xaxis={'categoryorder': 'total descending'})
fig.show()

fig_pie = px.pie(stable_sum, values='usd_value_millions', names='protocol', title='Share of Stablecoins by Protocol | '+tokens_str)
fig_pie.update_traces(textposition='inside', 
                        text = stable_sum['usd_value_millions'].map("${:,.2f}M".format),
                        textinfo='percent+label+text')
fig_pie.show()